**This comprehensive test notebook will:**
- Test all CRUD operations - upsert, search, delete

- Verify namespace functionality - separate data spaces

- Check performance - batch operations and search speed

- Handle edge cases - empty inputs, similar vectors, non-existent data

- Provide detailed reporting - success/failure status for each test

- Include cleanup - remove test data after tests

In [1]:
# test_firestore_vector_db.ipynb
import asyncio
import logging
import sys
import os
import numpy as np
from typing import List, Dict, Any

# Add the project root to Python path
sys.path.append('/app')

# Set up logging
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

# Import the local vector DB service
from src.services.vector_db.local_vector_db import LocalVectorDBService
from src.config import settings

# Initialize the service
vector_db = LocalVectorDBService()

print("🔥 Firestore Local Vector DB Test Suite")
print("=" * 50)

INFO:src.config:Loading ARANGODB_HOST: http://arangodb:8529
INFO:src.config:Loading USE_MOCK_SERVICES: true
INFO:src.services.vector_db.local_vector_db:Successfully connected to Firestore Emulator


🔥 Firestore Local Vector DB Test Suite


# Test 1: Basic Connection Test

In [2]:
# Test 1: Basic Connection and Collection Setup
async def test_connection():
    print("🧪 Test 1: Testing Firestore Connection...")
    try:
        stats = await vector_db.get_index_stats()
        print(f"✅ Connection successful! Current stats: {stats}")
        return True
    except Exception as e:
        print(f"❌ Connection failed: {e}")
        return False

# Run test
connection_ok = await test_connection()

🧪 Test 1: Testing Firestore Connection...
✅ Connection successful! Current stats: {'vectors_count': 23, 'namespaces': ['test_namespace', 'user310', 'user101', 'user007', 'user2', 'user009', 'user_123', 'public', 'user311'], 'shards_count': 1, 'collection_name': 'vectors'}


# Test 2: Generate Sample Embeddings

In [11]:
# Test 2: Generate Sample Embeddings
# CORRECTED: Generate Sample Embeddings Function
def generate_sample_embeddings(num_vectors: int = 5, dimension: int = 384) -> tuple:
    """Generate sample embeddings for testing - FIXED VERSION"""
    print(f"  Generating {num_vectors} sample embeddings...")
    
    # Generate random vectors (normalized for cosine similarity)
    vectors = []
    for i in range(num_vectors):
        vec = np.random.randn(dimension).astype(np.float32)
        vec = vec / np.linalg.norm(vec)  # Normalize
        vectors.append(vec.tolist())
    
    # Generate IDs and metadata - FIXED: Ensure same length
    ids = [f"test_doc_{i}" for i in range(num_vectors)]
    
    metadatas = []
    sample_titles = [
        "Machine Learning Research Paper",
        "Deep Learning Applications", 
        "Natural Language Processing",
        "Computer Vision Advances",
        "Reinforcement Learning"
    ]
    
    # FIXED: Generate metadata for ALL vectors, not just first 5
    for i in range(num_vectors):
        title = sample_titles[i % len(sample_titles)]  # Cycle through titles if num_vectors > 5
        metadatas.append({
            "title": f"{title} #{i}",
            "authors": [f"Author_{i}", f"Researcher_{i % 5}"],
            "year": 2020 + (i % 5),
            "abstract": f"This is a test abstract for {title} - document {i}",
            "doc_id": f"doc_{i}",
            "is_public": i % 2 == 0  # Alternate public/private
        })
    
    print(f"  ✅ Generated {len(vectors)} embeddings, {len(ids)} IDs, {len(metadatas)} metadata entries")
    return vectors, ids, metadatas


def generate_sample_embeddings_OLD(num_vectors: int = 5, dimension: int = 384) -> tuple:
    """Generate sample embeddings for testing"""
    print(f"\n🧪 Test 2: Generating {num_vectors} sample embeddings...")
    
    # Generate random vectors (normalized for cosine similarity)
    vectors = []
    for i in range(num_vectors):
        vec = np.random.randn(dimension).astype(np.float32)
        vec = vec / np.linalg.norm(vec)  # Normalize
        vectors.append(vec.tolist())
    
    # Generate IDs and metadata
    ids = [f"test_doc_{i}" for i in range(num_vectors)]
    
    metadatas = []
    sample_titles = [
        "Machine Learning Research Paper",
        "Deep Learning Applications", 
        "Natural Language Processing",
        "Computer Vision Advances",
        "Reinforcement Learning"
    ]
    
    for i, title in enumerate(sample_titles):
        metadatas.append({
            "title": title,
            "authors": [f"Author_{i}", f"Researcher_{i}"],
            "year": 2020 + i,
            "abstract": f"This is a test abstract for {title}",
            "doc_id": f"doc_{i}",
            "is_public": i % 2 == 0  # Alternate public/private
        })
    
    print(f"✅ Generated {len(vectors)} embeddings with dimension {len(vectors[0])}")
    return vectors, ids, metadatas

# Generate sample data
vectors, ids, metadatas = generate_sample_embeddings()
print(f"Sample vector: {vectors[0][:5]}...")  # Show first 5 dimensions
print(f"Sample metadata: {metadatas[0]}")

  Generating 5 sample embeddings...
  ✅ Generated 5 embeddings, 5 IDs, 5 metadata entries
Sample vector: [-0.031366072595119476, 0.014946346171200275, 0.004160337150096893, 0.06344591826200485, -0.01146529708057642]...
Sample metadata: {'title': 'Machine Learning Research Paper #0', 'authors': ['Author_0', 'Researcher_0'], 'year': 2020, 'abstract': 'This is a test abstract for Machine Learning Research Paper - document 0', 'doc_id': 'doc_0', 'is_public': True}


In [12]:
# Debug: Check lengths before running performance test
def debug_length_check():
    print("🔍 Debug: Checking vector generation...")
    
    # Test the original broken function
    def original_broken_function(num_vectors=20):
        vectors = [f"vector_{i}" for i in range(num_vectors)]
        ids = [f"id_{i}" for i in range(num_vectors)]
        
        metadatas = []
        sample_titles = ["Title1", "Title2", "Title3", "Title4", "Title5"]
        for i, title in enumerate(sample_titles):
            metadatas.append({"title": title})
            
        return vectors, ids, metadatas
    
    vectors, ids, metadatas = original_broken_function(20)
    print(f"❌ BROKEN: vectors: {len(vectors)}, ids: {len(ids)}, metadatas: {len(metadatas)}")
    
    # Test the fixed function
    def fixed_function(num_vectors=20):
        vectors = [f"vector_{i}" for i in range(num_vectors)]
        ids = [f"id_{i}" for i in range(num_vectors)]
        
        metadatas = []
        sample_titles = ["Title1", "Title2", "Title3", "Title4", "Title5"]
        for i in range(num_vectors):
            title = sample_titles[i % len(sample_titles)]
            metadatas.append({"title": f"{title}_{i}"})
            
        return vectors, ids, metadatas
    
    vectors, ids, metadatas = fixed_function(20)
    print(f"✅ FIXED: vectors: {len(vectors)}, ids: {len(ids)}, metadatas: {len(metadatas)}")

debug_length_check()

🔍 Debug: Checking vector generation...
❌ BROKEN: vectors: 20, ids: 20, metadatas: 5
✅ FIXED: vectors: 20, ids: 20, metadatas: 20


# Test 3: Upsert/Insert Test

In [4]:
# Test 3: Upsert/Insert Test
async def test_upsert():
    print(f"\n🧪 Test 3: Testing Upsert Operations...")
    
    # Test with different namespaces
    namespaces = ["user_123", "public", "test_namespace"]
    
    for namespace in namespaces:
        print(f"\n📦 Upserting to namespace: {namespace}")
        
        try:
            results = await vector_db.upsert_embeddings(
                vectors=vectors,
                ids=[f"{namespace}_{id}" for id in ids],  # Make IDs namespace-specific
                metadatas=metadatas,
                namespace=namespace
            ) 
            
            # Check results
            successes = [r for r in results if r["status"] == "success"]
            errors = [r for r in results if r["status"] == "error"]
            
            print(f"  ✅ Successfully upserted: {len(successes)} vectors")
            if errors:
                print(f"  ❌ Errors: {len(errors)}")
                for error in errors:
                    print(f"    - {error}")
            else:
                print(f"  ✅ No errors")
                
        except Exception as e:
            print(f"  ❌ Upsert failed for namespace {namespace}: {e}")
    
    # Check final stats
    stats = await vector_db.get_index_stats()
    print(f"\n📊 Final collection stats: {stats}")

# Run test
await test_upsert()


🧪 Test 3: Testing Upsert Operations...

📦 Upserting to namespace: user_123


INFO:src.services.vector_db.local_vector_db:Upserted batch of 5 vectors to Firestore
INFO:src.services.vector_db.local_vector_db:Upserted batch of 5 vectors to Firestore
INFO:src.services.vector_db.local_vector_db:Upserted batch of 5 vectors to Firestore


  ✅ Successfully upserted: 5 vectors
  ✅ No errors

📦 Upserting to namespace: public
  ✅ Successfully upserted: 5 vectors
  ✅ No errors

📦 Upserting to namespace: test_namespace
  ✅ Successfully upserted: 5 vectors
  ✅ No errors

📊 Final collection stats: {'vectors_count': 15, 'namespaces': ['test_namespace', 'user_123', 'public'], 'shards_count': 1, 'collection_name': 'vectors'}


# Test 4: Search Test

In [5]:
async def test_search():
    print(f"\n🧪 Test 4: Testing Search Operations...")
    
    # Use the first vector as query
    query_vector = vectors[0]
    
    # Test 4.1: Search across all namespaces
    print(f"\n🔍 Test 4.1: Search across all namespaces")
    try:
        results = await vector_db.search(
            query_embedding=query_vector,
            top_k=3,
            namespace=None  # Search all namespaces
        )
        
        print(f"  Found {len(results)} results:")
        for i, result in enumerate(results):
            print(f"    {i+1}. ID: {result['id']}, Distance: {result['distance']:.4f}")
            print(f"       Title: {result['metadata'].get('title', 'N/A')}")
            
    except Exception as e:
        print(f"  ❌ Search failed: {e}")
    
    # Test 4.2: Search within specific namespace
    print(f"\n🔍 Test 4.2: Search within 'user_123' namespace")
    try:
        results = await vector_db.search(
            query_embedding=query_vector,
            top_k=2,
            namespace="user_123"
        )
        
        print(f"  Found {len(results)} results in namespace 'user_123':")
        for i, result in enumerate(results):
            print(f"    {i+1}. ID: {result['id']}, Distance: {result['distance']:.4f}")
            
    except Exception as e:
        print(f"  ❌ Namespace search failed: {e}")
    
    # Test 4.3: Search with filters
    print(f"\n🔍 Test 4.3: Search with metadata filters")
    try:
        results = await vector_db.search(
            query_embedding=query_vector,
            top_k=5,
            namespace=None,
            filter={"is_public": [True]}  # Only public documents
        )
        
        print(f"  Found {len(results)} public documents:")
        for i, result in enumerate(results):
            is_public = result['metadata'].get('is_public', False)
            print(f"    {i+1}. ID: {result['id']}, Public: {is_public}")
            
    except Exception as e:
        print(f"  ❌ Filtered search failed: {e}")

# Run test
await test_search()


🧪 Test 4: Testing Search Operations...

🔍 Test 4.1: Search across all namespaces
  Found 3 results:
    1. ID: public_test_doc_0, Distance: 0.0000
       Title: Machine Learning Research Paper
    2. ID: test_namespace_test_doc_0, Distance: 0.0000
       Title: Machine Learning Research Paper
    3. ID: user_123_test_doc_0, Distance: 0.0000
       Title: Machine Learning Research Paper

🔍 Test 4.2: Search within 'user_123' namespace
  Found 2 results in namespace 'user_123':
    1. ID: user_123_test_doc_0, Distance: 0.0000
    2. ID: user_123_test_doc_4, Distance: 0.9278

🔍 Test 4.3: Search with metadata filters
  Found 0 public documents:


/usr/local/lib/python3.11/site-packages/google/cloud/firestore_v1/base_collection.py:304: UserWarning: Detected filter using positional arguments. Prefer using the 'filter' keyword argument instead.
  return query.where(field_path, op_string, value)


# Test 5: Delete Test

In [6]:
async def test_delete():
    print(f"\n🧪 Test 5: Testing Delete Operations...")
    
    # Delete documents from user_123 namespace
    docs_to_delete = [f"user_123_{id}" for id in ids[:2]]  # Delete first 2 docs
    
    print(f"🗑️  Deleting {len(docs_to_delete)} documents: {docs_to_delete}")
    
    try:
        delete_results = await vector_db.delete(docs_to_delete)
        
        successes = [r for r in delete_results if r["status"] == "success"]
        errors = [r for r in delete_results if r["status"] == "error"]
        
        print(f"  ✅ Successfully deleted: {len(successes)} documents")
        if errors:
            print(f"  ❌ Delete errors: {len(errors)}")
            for error in errors:
                print(f"    - {error}")
    
    except Exception as e:
        print(f"  ❌ Delete operation failed: {e}")
    
    # Verify deletion by searching
    print(f"\n🔍 Verifying deletion by searching...")
    try:
        query_vector = vectors[0]
        results = await vector_db.search(
            query_embedding=query_vector,
            top_k=10,
            namespace="user_123"
        )
        
        remaining_ids = [r["id"] for r in results]
        print(f"  Remaining documents in 'user_123': {len(remaining_ids)}")
        print(f"  Remaining IDs: {remaining_ids}")
        
        # Check if deleted docs are gone
        for deleted_id in docs_to_delete:
            if deleted_id in remaining_ids:
                print(f"  ⚠️  Document {deleted_id} still exists!")
            else:
                print(f"  ✅ Document {deleted_id} successfully deleted")
                
    except Exception as e:
        print(f"  ❌ Verification search failed: {e}")

# Run test
await test_delete()


🧪 Test 5: Testing Delete Operations...
🗑️  Deleting 2 documents: ['user_123_test_doc_0', 'user_123_test_doc_1']
  ✅ Successfully deleted: 2 documents

🔍 Verifying deletion by searching...
  Remaining documents in 'user_123': 3
  Remaining IDs: ['user_123_test_doc_4', 'user_123_test_doc_3', 'user_123_test_doc_2']
  ✅ Document user_123_test_doc_0 successfully deleted
  ✅ Document user_123_test_doc_1 successfully deleted


# Test 6: Performance Test

In [7]:
# Test 6: Performance Test - FIXED VERSION
async def test_performance():
    print(f"\n🧪 Test 6: Testing Performance...")
    
    # FIXED: Generate consistent number of vectors, ids, and metadatas
    def generate_large_sample_embeddings(num_vectors: int = 20, dimension: int = 384) -> tuple:
        """Generate sample embeddings for performance testing"""
        print(f"  Generating {num_vectors} sample embeddings...")
        
        # Generate random vectors (normalized for cosine similarity)
        vectors = []
        for i in range(num_vectors):
            vec = np.random.randn(dimension).astype(np.float32)
            vec = vec / np.linalg.norm(vec)  # Normalize
            vectors.append(vec.tolist())
        
        # Generate IDs and metadata - FIXED: Ensure same length
        ids = [f"test_doc_{i}" for i in range(num_vectors)]
        
        metadatas = []
        sample_titles = [
            "Machine Learning Research Paper",
            "Deep Learning Applications", 
            "Natural Language Processing",
            "Computer Vision Advances",
            "Reinforcement Learning",
            "Neural Networks Study",
            "AI Ethics and Society",
            "Quantum Computing Research",
            "Data Mining Techniques",
            "Robotics and Automation"
        ]
        
        # FIXED: Generate metadata for ALL vectors, not just first 5
        for i in range(num_vectors):
            title = sample_titles[i % len(sample_titles)]  # Cycle through titles
            metadatas.append({
                "title": f"{title} #{i}",
                "authors": [f"Author_{i}", f"Researcher_{i % 5}"],
                "year": 2020 + (i % 5),
                "abstract": f"This is a test abstract for {title} - document {i}",
                "doc_id": f"doc_{i}",
                "is_public": i % 2 == 0  # Alternate public/private
            })
        
        print(f"  ✅ Generated {len(vectors)} embeddings, {len(ids)} IDs, {len(metadatas)} metadata entries")
        return vectors, ids, metadatas

    # Generate larger batch with FIXED function
    large_vectors, large_ids, large_metadatas = generate_large_sample_embeddings(
        num_vectors=20, 
        dimension=384
    )
    
    print(f"⏱️  Testing with {len(large_vectors)} vectors...")
    
    import time
    
    # Test upsert performance
    start_time = time.time()
    results = await vector_db.upsert_embeddings(
        vectors=large_vectors,
        ids=[f"perf_test_{id}" for id in large_ids],
        metadatas=large_metadatas,
        namespace="performance_test"
    )
    upsert_time = time.time() - start_time
    
    successes = len([r for r in results if r["status"] == "success"])
    print(f"  📈 Upsert Performance: {successes} docs in {upsert_time:.2f}s "
          f"({successes/upsert_time:.1f} docs/sec)")
    
    # Test search performance
    start_time = time.time()
    search_results = await vector_db.search(
        query_embedding=large_vectors[0],
        top_k=10,
        namespace="performance_test"
    )
    search_time = time.time() - start_time
    
    print(f"  📈 Search Performance: {len(search_results)} results in {search_time:.4f}s")
    
    # Verify search results
    if search_results:
        print(f"  🔍 Best match: {search_results[0]['id']} (distance: {search_results[0]['distance']:.4f})")
    
    # Clean up performance test data
    delete_results = await vector_db.delete([f"perf_test_{id}" for id in large_ids])
    delete_successes = len([r for r in delete_results if r["status"] == "success"])
    print(f"  🧹 Cleaned up {delete_successes} performance test documents")

# Run test
await test_performance()


🧪 Test 6: Testing Performance...

🧪 Test 2: Generating 20 sample embeddings...
✅ Generated 20 embeddings with dimension 384
⏱️  Testing with 20 vectors...


ValueError: Vectors, IDs, and metadatas must have the same length

# Test 7: Edge Cases Test

In [8]:
# Test 7: Edge Cases Test
async def test_edge_cases():
    print(f"\n🧪 Test 7: Testing Edge Cases...")
    
    # Test 7.1: Empty inputs
    print(f"\n🔍 Test 7.1: Empty inputs")
    try:
        results = await vector_db.upsert_embeddings([], [], [])
        print(f"  ✅ Empty upsert handled: {results}")
    except Exception as e:
        print(f"  ❌ Empty upsert failed: {e}")
    
    # Test 7.2: Very similar vectors
    print(f"\n🔍 Test 7.2: Similar vectors search")
    try:
        # Create two very similar vectors
        base_vector = np.random.randn(384).astype(np.float32)
        base_vector = base_vector / np.linalg.norm(base_vector)
        
        similar_vector = base_vector + 0.01 * np.random.randn(384)
        similar_vector = similar_vector / np.linalg.norm(similar_vector)
        
        # Store them
        await vector_db.upsert_embeddings(
            vectors=[base_vector.tolist(), similar_vector.tolist()],
            ids=["similar_1", "similar_2"],
            metadatas=[{"test": "similar_1"}, {"test": "similar_2"}],
            namespace="similarity_test"
        )
        
        # Search and check distance
        results = await vector_db.search(
            query_embedding=base_vector.tolist(),
            top_k=2,
            namespace="similarity_test"
        )
        
        print(f"  Similar vectors distance: {results[1]['distance']:.6f}")
        print(f"  ✅ Similar vectors test completed")
        
        # Clean up
        await vector_db.delete(["similar_1", "similar_2"])
        
    except Exception as e:
        print(f"  ❌ Similar vectors test failed: {e}")
    
    # Test 7.3: Non-existent namespace search
    print(f"\n🔍 Test 7.3: Non-existent namespace")
    try:
        results = await vector_db.search(
            query_embedding=vectors[0],
            top_k=5,
            namespace="non_existent_namespace_12345"
        )
        print(f"  Search in non-existent namespace returned: {len(results)} results")
        print(f"  ✅ Non-existent namespace handled correctly")
    except Exception as e:
        print(f"  ❌ Non-existent namespace search failed: {e}")

# Run test
await test_edge_cases()

INFO:src.services.vector_db.local_vector_db:Upserted batch of 2 vectors to Firestore



🧪 Test 7: Testing Edge Cases...

🔍 Test 7.1: Empty inputs
  ✅ Empty upsert handled: []

🔍 Test 7.2: Similar vectors search
  Similar vectors distance: 0.018226
  ✅ Similar vectors test completed

🔍 Test 7.3: Non-existent namespace
  Search in non-existent namespace returned: 0 results
  ✅ Non-existent namespace handled correctly


# Final Summary and Cleanup

In [9]:
# Final Summary and Cleanup
async def final_summary():
    print(f"\n" + "="*50)
    print(f"📊 FINAL TEST SUMMARY")
    print(f"="*50)
    
    # Get final stats
    stats = await vector_db.get_index_stats()
    print(f"Final collection stats: {stats}")
    
    # Count documents by namespace
    print(f"\n📂 Documents by namespace:")
    namespaces = ["user_123", "public", "test_namespace", "performance_test", "similarity_test"]
    
    for namespace in namespaces:
        try:
            results = await vector_db.search(
                query_embedding=vectors[0],
                top_k=100,  # Large number to count all
                namespace=namespace
            )
            print(f"  {namespace}: {len(results)} documents")
        except:
            print(f"  {namespace}: 0 documents (or error)")
    
    print(f"\n🎯 Test Suite Completed!")
    print(f"✅ Firestore Local Vector DB is working correctly")
    print(f"💡 Use VECTOR_DB_TYPE=local in your environment to enable this service")

# Run final summary
await final_summary()


📊 FINAL TEST SUMMARY
Final collection stats: {'vectors_count': 13, 'namespaces': ['test_namespace', 'user_123', 'public'], 'shards_count': 1, 'collection_name': 'vectors'}

📂 Documents by namespace:
  user_123: 3 documents
  public: 5 documents
  test_namespace: 5 documents
  performance_test: 0 documents
  similarity_test: 0 documents

🎯 Test Suite Completed!
✅ Firestore Local Vector DB is working correctly
💡 Use VECTOR_DB_TYPE=local in your environment to enable this service


# Quick health check function for future use

In [13]:
# Quick health check function for future use
async def health_check():
    """Quick health check for the vector DB"""
    try:
        stats = await vector_db.get_index_stats()
        print(f"🟢 Vector DB Healthy - Total vectors: {stats.get('vectors_count', 0)}")
        return True
    except Exception as e:
        print(f"🔴 Vector DB Unhealthy: {e}")
        return False

# Run quick health check
print(f"\n🏥 Running Quick Health Check...")
is_healthy = await health_check()


🏥 Running Quick Health Check...
🟢 Vector DB Healthy - Total vectors: 13


# Miscs

In [66]:
await vector_db.get_index_stats()

{'vectors_count': 18,
 'namespaces': ['public', 'test_namespace', 'user_123', 'user007'],
 'shards_count': 1,
 'collection_name': 'vectors'}

In [15]:
help(vector_db.upsert_embeddings)

Help on method upsert_embeddings in module src.services.vector_db.local_vector_db:

async upsert_embeddings(vectors: List[List[float]], ids: List[str], metadatas: List[Dict[str, Any]], namespace: Optional[str] = None) method of src.services.vector_db.local_vector_db.LocalVectorDBService instance
    Upsert embeddings into Firestore with vector support.



In [51]:
await vector_db.upsert_embeddings(vectors=[[.35]*384,], 
                                  ids=['abc5',], 
                                  metadatas=[{'a': 2, 'b': 3}], 
                                  namespace='user007')

INFO:src.services.vector_db.local_vector_db:Upserted batch of 1 vectors to Firestore


[{'status': 'success', 'id': 'abc5'}]

In [21]:
await vector_db.get_index_stats()

{'vectors_count': 21,
 'namespaces': ['test_namespace',
  'user009',
  'user2',
  'user310',
  'user007',
  'user_123',
  'public'],
 'shards_count': 1,
 'collection_name': 'vectors'}

In [53]:
help(vector_db.search)

Help on method search in module src.services.vector_db.local_vector_db:

async search(query_embedding: List[float], top_k: int, namespace: Optional[str] = None, filter: Optional[Dict[str, Any]] = None) -> List[Dict[str, Any]] method of src.services.vector_db.local_vector_db.LocalVectorDBService instance
    Search for similar vectors using cosine similarity.



In [64]:
await vector_db.search(query_embedding=[0.35]*384,
                       top_k=2,
                       namespace="user007")

[{'id': 'abc5',
  'distance': np.float64(1.1102230246251565e-16),
  'metadata': {'b': 3, 'a': 2}},
 {'id': 'abc1',
  'distance': np.float64(2.220446049250313e-16),
  'metadata': {'b': 3, 'a': 2}}]

In [22]:
await vector_db.search(query_embedding=[1]*384,
                       top_k=10,
                       namespace="user310")

/usr/local/lib/python3.11/site-packages/google/cloud/firestore_v1/base_collection.py:304: UserWarning: Detected filter using positional arguments. Prefer using the 'filter' keyword argument instead.
  return query.where(field_path, op_string, value)


[{'id': 'user310_b7911978-2c06-4061-8dbf-5050f46f19d3',
  'distance': np.float64(0.9873289903297738),
  'metadata': {'publication_date': None,
   'user_id': 'user310',
   'authors': [],
   'doc_id': 'b7911978-2c06-4061-8dbf-5050f46f19d3',
   'namespace': 'user310',
   'text_preview': '©          Nature Publishing Group\n1953\nNo. 4356\nApril 25, 1953\nNATURE\n737\nThis\nfigure\nis\npurely\ndiagrammatic. The two\nribbons symbolize the\ntwo\nphosphate-sugar\nchains, and the hori-\nzontal rods the p...',
   'filename': 'WatsonCrick1953.pdf',
   'journal': None,
   'is_public': False,
   'title': 'WatsonCrick1953.pdf'}}]

In [4]:
query_vector =  [-0.08533287048339844, 0.06320766359567642, -0.05387697368860245, -0.012143664062023163, -0.039499204605817795, 0.007550196722149849, -0.010784433223307133, 0.02410038374364376, -0.012486048974096775, -0.0017627258785068989, 0.02419915236532688, 0.05066535621881485, -0.07333450019359589, 0.004249769728630781, -0.08437206596136093, 0.06910083442926407, -0.09553465247154236, 0.03943271562457085, 0.019373342394828796, -0.06292670965194702, -0.04554183781147003, 0.01868239976465702, 0.07621124386787415, -0.011143232695758343, -0.012767351232469082, 0.07841664552688599, -0.002308505354449153, 0.010973022319376469, 0.03595571219921112, -0.053781650960445404, 0.07792655378580093, -0.013682946562767029, 0.020831771194934845, -0.011518816463649273, -0.016729161143302917, 0.018863700330257416, 0.05099675804376602, 0.03931378200650215, 0.011224928312003613, -0.02155255153775215, 0.0655590072274208, 0.006225470453500748, -0.026553748175501823, 0.13485614955425262, -0.04091698303818703, 0.06857798993587494, 0.05803070217370987, 0.048606015741825104, -0.03198649361729622, 0.02669346332550049, -0.007402851711958647, 0.03202458843588829, -0.01701812632381916, -0.025494588539004326, -0.004538432694971561, 0.0992225930094719, 0.0031316494569182396, -0.11517127603292465, -0.0016264505684375763, 0.022243130952119827, 0.08529284596443176, -0.05047579109668732, 4.5627442887052894e-05, 0.0009597937460057437, 0.11981269717216492, -0.005669879261404276, 0.07470890879631042, -0.03762474283576012, -0.016144007444381714, -0.06479991227388382, 0.017802894115447998, -0.017281800508499146, 0.03861672058701515, 0.048282917588949203, 0.008790770545601845, 0.007283375132828951, -0.03435134142637253, -0.055734820663928986, -0.019888821989297867, 0.1076444610953331, -0.058844320476055145, -0.07172531634569168, 0.006923135835677385, 0.023207608610391617, -0.011990099214017391, -0.03280875086784363, 0.06460632383823395, 0.03647578880190849, -0.036100614815950394, 0.0814611166715622, -0.09594447165727615, -0.026434509083628654, 0.069469153881073, -0.05066205933690071, -0.09883996099233627, 0.04510064795613289, 0.026256700977683067, -0.002860547974705696, 0.08153462409973145, 0.011127176694571972, -0.011633064597845078, -0.04997675493359566, -0.0752483382821083, -0.016100700944662094, -0.039464615285396576, -0.0204386655241251, -0.018704883754253387, -0.051259901374578476, 0.04636237397789955, -0.00139285356272012, -0.09742899984121323, 0.03802228718996048, -0.08427175879478455, 0.01753644086420536, -0.054727889597415924, -0.07330755889415741, 0.05590727925300598, -0.0023529776372015476, 0.024532394483685493, 0.010527241043746471, 0.02014165371656418, -0.01898735947906971, 0.029375987127423286, 0.003994782455265522, 0.01221164409071207, -0.04970862343907356, -0.024886833503842354, -9.963653217410886e-33, -0.023432744666934013, -0.023082146421074867, 0.023603731766343117, 0.01257326640188694, -0.013141063041985035, 0.05215912684798241, -0.01931684836745262, 0.011101225391030312, -0.14605362713336945, 0.02777140773832798, -0.023429110646247864, -0.061470430344343185, 0.07261207699775696, 0.026576748117804527, -0.027827071025967598, 0.07294709980487823, -0.09049038589000702, -0.0012167638633400202, -0.008118044584989548, -0.0159293320029974, -0.03101527877151966, 0.029244573786854744, -0.0053628915920853615, 0.008256728760898113, 0.020235523581504822, -0.0028019710443913937, -0.04873496666550636, -0.006920344196259975, 0.12933507561683655, -0.03802759200334549, -0.006026070099323988, -0.030194710940122604, 0.03016921691596508, 0.04274723678827286, 0.08155557513237, 0.06508582085371017, 0.04461285099387169, -0.03456072509288788, 0.049432288855314255, 0.10238329321146011, -0.048643313348293304, -0.05536014214158058, 0.029778340831398964, -0.07861531525850296, 0.04019642248749733, -0.007291090674698353, 0.007094314321875572, -0.12434857338666916, -0.1599123477935791, 0.004601133055984974, -0.020431403070688248, -0.06117594242095947, 0.007727866526693106, 0.0012544011697173119, -0.030100857838988304, 0.0111935343593359, -0.03409412503242493, 0.019648917019367218, -0.06251687556505203, 0.040291015058755875, -0.016211966052651405, 0.046666208654642105, -0.03312103450298309, 0.002359357662498951, 0.08046545088291168, -0.06217731162905693, 0.011035485193133354, -0.011882679536938667, 0.1113336831331253, 0.002472342923283577, 0.08270048350095749, -0.05266692489385605, -0.0043308185413479805, 0.012113873846828938, 0.016892649233341217, 0.07485593110322952, -0.08653387427330017, -0.020224757492542267, -0.09040670096874237, 0.06816992908716202, -0.05454012006521225, -0.025298919528722763, 0.011182673275470734, -0.06019110232591629, -0.02348862588405609, 0.04027552157640457, -0.08663056790828705, -0.03798661008477211, 0.027609484270215034, -0.028922822326421738, 0.03741922974586487, -0.07406778633594513, -0.007277278695255518, -0.14549526572227478, 0.04574810713529587, 4.241780915833825e-33, 0.05708735063672066, -0.08079569041728973, 0.01966448500752449, 0.06122260168194771, 0.0007716532563790679, -0.0999399721622467, 0.05183149874210358, 0.11116880923509598, 0.001987385330721736, 0.013450060971081257, 0.043279845267534256, -0.056606799364089966, 0.01727987453341484, -0.020304877310991287, 0.013308452442288399, 0.01634853519499302, 0.0019935264717787504, 0.03884073346853256, 0.04412056878209114, 0.045426711440086365, -0.019499365240335464, 0.0788140818476677, -0.031042061746120453, 0.030786506831645966, 0.012223545461893082, -0.037807147949934006, -0.0457841195166111, 0.013871788047254086, 0.07482399046421051, -0.03833424672484398, -0.1233457699418068, 0.010708640329539776, 0.0807511955499649, 0.00046469864901155233, 0.06227744743227959, 0.03855273500084877, 0.07165703922510147, 0.057791732251644135, 0.030867379158735275, -0.031994692981243134, -0.0881940945982933, 0.10427526384592056, -0.07915260642766953, 0.009461699053645134, -0.007622865028679371, 0.04222763702273369, -0.015836700797080994, 0.09821775555610657, -0.025047294795513153, 5.660894385073334e-05, -0.009490152820944786, 0.01862039417028427, 0.009072774089872837, -0.07719580829143524, 0.01952219009399414, -0.034877851605415344, -0.004846116062253714, -0.03641827777028084, 0.045342739671468735, 0.11328764259815216, 0.07033498585224152, -0.03148656338453293, -0.021632522344589233, 0.005436321720480919, -0.018396146595478058, 0.055580999702215195, -0.03274228051304817, 0.02729525975883007, -0.01347374077886343, -0.027900414541363716, 0.03442836180329323, 0.009451204910874367, 0.034716345369815826, 0.03854938969016075, 0.016596874222159386, -0.04870692640542984, -0.07304654270410538, -0.008290898986160755, 0.019057398661971092, -0.00421536760404706, 0.013417547568678856, -0.08433642983436584, -0.020298711955547333, 0.02607601508498192, 0.11783704161643982, -0.02295668236911297, 0.021393686532974243, 0.04902345687150955, -0.05231544002890587, -0.055870190262794495, -0.02093527652323246, 0.031365443021059036, -0.05577091872692108, 0.03280707821249962, -0.03693254292011261, -1.5461541380545896e-08, 0.08316486328840256, -0.0859232246875763, 0.0017912626499310136, -0.09699346125125885, 0.042070385068655014, 0.06975790858268738, 0.017771778628230095, 0.007324211299419403, 0.060239873826503754, -0.016933942213654518, 0.004137400537729263, 0.06167007237672806, -0.046822916716337204, 0.0353974811732769, -0.003064175834879279, 0.015715697780251503, -0.019847821444272995, -0.05099118873476982, 0.0046591078862547874, 0.0524381585419178, 0.01329086534678936, 0.041092369705438614, -0.15533185005187988, 0.04834933951497078, 0.013248312287032604, 0.05052652955055237, 0.06338553130626678, 0.02509012632071972, 0.015886686742305756, -0.038277339190244675, -0.013269362971186638, -0.051046278327703476, 0.021145107224583626, 0.047841303050518036, -0.05881962180137634, -0.04995081573724747, 0.0675315111875534, -0.0347600020468235, 0.008880374021828175, -0.014999696053564548, 0.037935771048069, 0.08151311427354813, -0.06835751980543137, -0.029214495792984962, -0.012053891085088253, -0.024773186072707176, 0.009407216683030128, -0.00875867996364832, -0.0579652301967144, -0.04436656832695007, -0.00439862534403801, -0.051215723156929016, -0.0263687577098608, -0.05069563910365105, -0.02624211646616459, -0.02586251311004162, 0.010995126329362392, 0.01454547792673111, -0.08349516242742538, -0.02024334855377674, 0.04096848890185356, 0.04112539067864418, 0.13260547816753387, -0.08521678298711777]

In [5]:
await vector_db.search(query_embedding=query_vector,
                       top_k=10,
                       namespace="user101")

/usr/local/lib/python3.11/site-packages/google/cloud/firestore_v1/base_collection.py:304: UserWarning: Detected filter using positional arguments. Prefer using the 'filter' keyword argument instead.
  return query.where(field_path, op_string, value)


[{'id': 'user101_9689da65-8b5d-43fb-b405-61ef1def3baa',
  'distance': np.float64(0.696872871315227),
  'metadata': {'journal': None,
   'user_id': 'user101',
   'doc_id': '9689da65-8b5d-43fb-b405-61ef1def3baa',
   'is_public': False,
   'authors': [],
   'title': 'WatsonCrick1953.pdf',
   'namespace': 'user101',
   'filename': 'WatsonCrick1953.pdf',
   'publication_date': None,
   'text_preview': '©          Nature Publishing Group\n1953\nNo. 4356\nApril 25, 1953\nNATURE\n737\nThis\nfigure\nis\npurely\ndiagrammatic. The two\nribbons symbolize the\ntwo\nphosphate-sugar\nchains, and the hori-\nzontal rods the p...'}}]

In [7]:
results = await vector_db.search(query_embedding=query_vector, 
                       top_k=10,
                       namespace="user310", 
                       filter={"metadata.is_public": [True,
                                  False]})  

In [8]:
results

[{'id': 'user310_b7911978-2c06-4061-8dbf-5050f46f19d3',
  'distance': np.float64(0.696872871315227),
  'metadata': {'journal': None,
   'user_id': 'user310',
   'authors': [],
   'is_public': False,
   'doc_id': 'b7911978-2c06-4061-8dbf-5050f46f19d3',
   'title': 'WatsonCrick1953.pdf',
   'namespace': 'user310',
   'filename': 'WatsonCrick1953.pdf',
   'publication_date': None,
   'text_preview': '©          Nature Publishing Group\n1953\nNo. 4356\nApril 25, 1953\nNATURE\n737\nThis\nfigure\nis\npurely\ndiagrammatic. The two\nribbons symbolize the\ntwo\nphosphate-sugar\nchains, and the hori-\nzontal rods the p...'}}]

In [9]:

    async def _build_context(relevant_papers: List[Dict[str, Any]],
                             query_text: str) -> str:
        """Build context from relevant papers for the LLM"""
        context_parts = []

        for i, paper in enumerate(
                relevant_papers[:5]):  # Limit to top 5 papers for context
            metadata = paper.get('metadata', {})
            doc_id = metadata.get('doc_id')

            if not doc_id:
                continue

            # In a real implementation, you would retrieve the full text from storage
            # For now, we'll use the metadata and a placeholder for text
            context_parts.append(
                f"Paper {i+1}: {metadata.get('title', 'Unknown title')}\n"
                f"Authors: {', '.join(metadata.get('authors', []))}\n"
                f"Abstract: {metadata.get('abstract', 'No abstract available')}\n"
                f"Relevance score: {paper.get('score', 0):.3f}\n")

        if not context_parts:
            return "No relevant research papers found for this query."

        return "\n".join(context_parts)


In [24]:
from typing import Optional


async def _build_context(relevant_papers: List[Dict[str, Any]],
                         query_text: str) -> str:
    """Build context from relevant papers for the LLM by retrieving full text"""
    context_parts = []
    
    # Get storage service (add this to your __init__ or get it when needed)
    from src.services.storage import get_storage_service
    storage_service = get_storage_service()

    for i, paper in enumerate(relevant_papers[:5]):  # Limit to top 5 papers
        metadata = paper.get('metadata', {})
        doc_id = metadata.get('doc_id')

        if not doc_id:
            continue

        try:
            # Try to get the full text from storage
            
            full_text = await _get_paper_full_text(metadata, storage_service)
            print("*="*30)
            print("*="*30)
            print(f"full text length {len(full_text)}")
            print("*="*30)
            print("*="*30)
            
            # Use full text if available, otherwise fall back to abstract/preview
            text_content = full_text if full_text else metadata.get('abstract', metadata.get('text_preview', 'No content available'))
            
            # Truncate text to reasonable length for LLM context
            if len(text_content) > 2000:
                text_content = text_content[:2000] + "... [truncated]"
            
            context_parts.append(
                f"Paper {i+1}: {metadata.get('title', 'Unknown title')}\n"
                f"Authors: {', '.join(metadata.get('authors', []))}\n"
                f"Content: {text_content}\n"
                f"Relevance score: {1 - paper.get('distance', 1):.3f}\n"  # Convert distance to similarity
            )
            
        except Exception as e:
            logger.warning(f"Failed to retrieve full text for paper {doc_id}: {e}")
            # Fallback to metadata-only approach
            context_parts.append(
                f"Paper {i+1}: {metadata.get('title', 'Unknown title')}\n"
                f"Authors: {', '.join(metadata.get('authors', []))}\n"
                f"Abstract: {metadata.get('abstract', 'No abstract available')}\n"
                f"Preview: {metadata.get('text_preview', 'No preview available')}\n"
                f"Relevance score: {1 - paper.get('distance', 1):.3f}\n"
            )

    if not context_parts:
        return "No relevant research papers found for this query."

    return "\n".join(context_parts)

async def _get_paper_full_text(metadata: Dict[str, Any], storage_service) -> Optional[str]:
    """Retrieve the full text of a paper from storage"""
    try:
        # Method 1: Check if we have raw_text_path in metadata (from vector DB)
        raw_text_path = metadata.get('raw_text_path')
        if raw_text_path:
            text_content = await storage_service.download_file(raw_text_path)
            return text_content.decode('utf-8') if isinstance(text_content, bytes) else text_content

        print("*="*30)
        print(f"txt content: {len(text_content)}")
        print(f"{metadata}")
        print("*="*30)
        
        # Method 2: Try to reconstruct the path from doc_id and user_id
        doc_id = metadata.get('doc_id')
        user_id = metadata.get('user_id')
        if doc_id and user_id:
            # Reconstruct the path pattern used in file_processing.py
            possible_paths = [
                f"users/{user_id}/text/{doc_id}.txt",
                f"users/{user_id}/text/{doc_id}",
                f"text/{doc_id}.txt"
            ]
            
            for path in possible_paths:
                try:
                    if await storage_service.file_exists(path):
                        text_content = await storage_service.download_file(path)
                        return text_content.decode('utf-8') if isinstance(text_content, bytes) else text_content
                except:
                    continue
        
        # Method 3: For production Vertex AI, we might need to query the paper database
        # This would require additional paper DB service integration
        logger.debug(f"Could not find full text for paper {metadata.get('doc_id')}")
        return None
        
    except Exception as e:
        logger.warning(f"Error retrieving full text for paper: {e}")
        return None

In [25]:
cnxt = await _build_context(results, 'dna')

INFO:src.services.storage:Using Mock Storage Service (development mode)
INFO:src.services.storage.mock_storage:Initializing Mock Storage Service for development


*=*=*=*=*=*=*=*=*=*=*=*=*=*=*=*=*=*=*=*=*=*=*=*=*=*=*=*=*=*=
*=*=*=*=*=*=*=*=*=*=*=*=*=*=*=*=*=*=*=*=*=*=*=*=*=*=*=*=*=*=
*=*=*=*=*=*=*=*=*=*=*=*=*=*=*=*=*=*=*=*=*=*=*=*=*=*=*=*=*=*=


In [15]:
len(cnxt)

275

In [28]:
p1 = results[0]

In [31]:
p1['metadata']

{'journal': None,
 'user_id': 'user310',
 'authors': [],
 'is_public': False,
 'doc_id': 'b7911978-2c06-4061-8dbf-5050f46f19d3',
 'title': 'WatsonCrick1953.pdf',
 'namespace': 'user310',
 'filename': 'WatsonCrick1953.pdf',
 'publication_date': None,
 'text_preview': '©          Nature Publishing Group\n1953\nNo. 4356\nApril 25, 1953\nNATURE\n737\nThis\nfigure\nis\npurely\ndiagrammatic. The two\nribbons symbolize the\ntwo\nphosphate-sugar\nchains, and the hori-\nzontal rods the p...'}